# CSVS to Hepfile
Imagine we have a database like structure with multiple csvs all connected by a common ID. See the household exmaple here: https://hepfile.readthedocs.io/en/latest/fundamentals.html#a-toy-example

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import awkward as ak
import hepfile as hf

In [ ]:

#datapath = os.path.join('/', 'home', 'nfranz', 'research', 'hepfile', 'docs', 'examples', 'example_nb', '*.csv')
datapath = os.path.join('/', 'home', 'bellis', 'hepfile', 'docs', 'examples', 'example_nb', '*.csv')

print(datapath)

files = glob.glob(datapath)
common_key = 'Household ID'
group_names = ['Residences', 'People', 'Vehicles']
awk = hf.csv_tools.csv_to_hepfile(files, common_key, outfile='test-csv.h5', group_names=group_names)

In [ ]:
data, _ = hf.load('test-csv.h5', return_awkward=True)

data